In [0]:

import os
import sys
import time
import math

import torch
import torch.nn as nn
import torch.nn.init as init


def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(3):
            mean[i] += inputs[:,i,:,:].mean()
            std[i] += inputs[:,i,:,:].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

def init_params(net):
    '''Init layer parameters.'''
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            init.kaiming_normal(m.weight, mode='fan_out')
            if m.bias:
                init.constant(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            init.constant(m.weight, 1)
            init.constant(m.bias, 0)
        elif isinstance(m, nn.Linear):
            init.normal(m.weight, std=1e-3)
            if m.bias:
                init.constant(m.bias, 0)


term_width = 80

TOTAL_BAR_LENGTH = 86.
last_time = time.time()
begin_time = last_time
def progress_bar(current, total, msg=None):
    global last_time, begin_time
    if current == 0:
        begin_time = time.time()  # Reset for new bar.

    cur_len = int(TOTAL_BAR_LENGTH*current/total)
    rest_len = int(TOTAL_BAR_LENGTH - cur_len) - 1

    sys.stdout.write(' [')
    for i in range(cur_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    cur_time = time.time()
    step_time = cur_time - last_time
    last_time = cur_time
    tot_time = cur_time - begin_time

    L = []
    L.append('  Step: %s' % format_time(step_time))
    L.append(' | Tot: %s' % format_time(tot_time))
    if msg:
        L.append(' | ' + msg)

    msg = ''.join(L)
    sys.stdout.write(msg)
    for i in range(term_width-int(TOTAL_BAR_LENGTH)-len(msg)-3):
        sys.stdout.write(' ')

    # Go back to the center of the bar.
    for i in range(term_width-int(TOTAL_BAR_LENGTH/2)):
        sys.stdout.write('\b')
    sys.stdout.write(' %d/%d ' % (current+1, total))

    if current < total-1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()

def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds*1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f


In [0]:
#Data

from __future__ import print_function
import argparse
import csv
import os
import sys
import numpy as np
import random
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)
random.seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

sys.argv = ['train.py', '--lr=0.1', '--seed=0', '--decay=1e-4']

parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--resume', '-r', action='store_true',
                    help='resume from checkpoint')
parser.add_argument('--model', default="ResNet18", type=str,
                    help='model type (default: ResNet18)')
parser.add_argument('--name', default='0', type=str, help='name of run')
parser.add_argument('--seed', default=0, type=int, help='random seed')
parser.add_argument('--batch-size', default=128, type=int, help='batch size')
parser.add_argument('--epoch', default=200, type=int,
                    help='total epochs to run')
parser.add_argument('--no-augment', dest='augment', action='store_false',
                    help='use standard augmentation (default: True)')
parser.add_argument('--decay', default=1e-4, type=float, help='weight decay')
parser.add_argument('--alpha', default=1., type=float,
                    help='mixup interpolation coefficient (default: 1)')
parser.add_argument('--beta', default=1, type=float,
                    help='hyperparameter beta')
parser.add_argument('--cutmix_prob', default=0.5, type=float,
                    help='cutmix probability')
args = parser.parse_args()

use_cuda = torch.cuda.is_available()

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

if args.seed != 0:
    torch.manual_seed(args.seed)

# Data
print('==> Preparing data..')
if args.augment:
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
    ])
else:
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
    ])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='~/data', train=True, download=True,
                            transform=transform_train)

percentage = 10
each = (5000 * percentage/100)
x = list(range(0, len(trainset), 1))

countsets = {}
for i in range(len(trainset)) :
    c = trainset.__getitem__(i)[1]
    if str(c) in countsets :
        if len(countsets[str(c)]) < each :
            countsets[str(c)].append(i)
    else :
        countsets[str(c)] = []
        countsets[str(c)].append(i)

labelled_indices = []
unlabelled_indices = []
for key in countsets :
    labelled_indices = labelled_indices + countsets[key]
    
print(len(labelled_indices))

unlabelled_indices = list(set(x) - set(labelled_indices))

labelled_data = torch.utils.data.Subset(trainset, labelled_indices)
unlabelled_data = torch.utils.data.Subset(trainset, unlabelled_indices)

trainloader_labelled = torch.utils.data.DataLoader(labelled_data,
                                          batch_size=256,
                                          shuffle=True, num_workers=8)

trainloader_unlabelled = torch.utils.data.DataLoader(unlabelled_data,
                                          batch_size=256,
                                          shuffle=True, num_workers=8)


testset = datasets.CIFAR10(root='~/data', train=False, download=False,
                           transform=transform_test)


x = list(range(0, len(testset), 1))
val_indices = list(range(0, len(testset), 2))
test_indices = list(set(x) - set(val_indices))
valsubset = torch.utils.data.Subset(testset, val_indices)
testsubset = torch.utils.data.Subset(testset, test_indices)

testsubset = torch.utils.data.Subset(testset, test_indices)

valloader = torch.utils.data.DataLoader(valsubset, batch_size=256,
                                         shuffle=False, num_workers=8)

testloader = torch.utils.data.DataLoader(testsubset, batch_size=256,
                                         shuffle=False, num_workers=8)


==> Preparing data..
Files already downloaded and verified
5000


In [0]:
def validate(testdataloader, test_net, epoch):
    global best_acc
    test_net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testdataloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        outputs = test_net(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.data
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        progress_bar(batch_idx, len(testloader),
                     'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (test_loss/(batch_idx+1), 100.*correct/total,
                        correct, total))
    acc = 100.*correct/total
    if epoch == start_epoch + args.epoch - 1 or acc > best_acc:
        checkpoint(test_net, acc, epoch)
    if acc > best_acc:
        best_acc = acc
    return (test_loss/batch_idx, 100.*correct/total)


def checkpoint(test_net, acc, epoch):
    # Save checkpoint.
    print('Saving..')
    state = {
        'net': test_net,
        'acc': acc,
        'epoch': epoch,
        'rng_state': torch.get_rng_state()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/ckpt.t7' + args.name + '_'
               + str(args.seed))


def adjust_learning_rate(optimizer, epoch):
    """decrease the learning rate at 100 and 150 epoch"""
    lr = args.lr
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [0]:
#Baseline
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.t7' + args.name + '_'
                            + str(args.seed))
    base_net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch'] + 1
    rng_state = checkpoint['rng_state']
    torch.set_rng_state(rng_state)
else:
    print('==> Building model..')
    base_net = models.resnet18()

if not os.path.isdir('results'):
    os.mkdir('results')
logname = ('results/log_baseline_train' + base_net.__class__.__name__ + '_' + args.name + '_'
           + str(args.seed) + '.csv')

if use_cuda:
    base_net.cuda()
    base_net = torch.nn.DataParallel(base_net)
    print(torch.cuda.device_count())
    cudnn.benchmark = True
    print('Using CUDA..')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(base_net.parameters(), lr=args.lr, momentum=0.9,
                      weight_decay=args.decay)

def baseline_train(net, epoch):
    print('\nEpoch: %d' % epoch)
    base_net.train()
    train_loss = 0
    reg_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader_labelled):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()

        outputs = base_net(inputs)
        loss = criterion(outputs, targets)
        train_loss += loss.data
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)

        correct += predicted.eq(targets.data).cpu().sum()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        progress_bar(batch_idx, len(trainloader_labelled),
                     'Loss: %.3f | Reg: %.5f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), reg_loss/(batch_idx+1),
                        100.*correct/total, correct, total))
    return (train_loss/batch_idx, reg_loss/batch_idx, 100.*correct/total)

if not os.path.exists(logname):
    with open(logname, 'w') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow(['epoch', 'train loss', 'reg loss', 'train acc',
                            'test loss', 'test acc'])

for epoch in range(start_epoch, args.epoch):
    train_loss, reg_loss, train_acc = baseline_train(base_net, epoch)
    test_loss, test_acc = validate(valloader, base_net, epoch)
    adjust_learning_rate(optimizer, epoch)
    with open(logname, 'a') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow([epoch, train_loss, reg_loss, train_acc, test_loss,
                            test_acc])

if not os.path.isdir('models'):
    os.mkdir('models')
torch.save(base_net, './models/baseline_train.pth')

==> Building model..
1
Using CUDA..

Epoch: 0
 [=================================================================================>....]  Step: 108ms | Tot: 3s200ms | Loss: 4.540 | Reg: 0.00000 | Acc: 13.840% (692/5000) 20/20 


train.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


 [=================================================================================>....]  Step: 27ms | Tot: 995ms | Loss: 1370.856 | Acc: 10.240% (512/5000) 20/20 
Saving..

Epoch: 1


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DataParallel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python

 [=================================================================================>....]  Step: 105ms | Tot: 3s67ms | Loss: 2.732 | Reg: 0.00000 | Acc: 19.420% (971/5000) 20/20 
 [=================================================================================>....]  Step: 30ms | Tot: 1s19ms | Loss: 5.909 | Acc: 17.940% (897/5000) 20/20 
Saving..

Epoch: 2
 [=================================================================================>....]  Step: 100ms | Tot: 3s51ms | Loss: 2.413 | Reg: 0.00000 | Acc: 23.920% (1196/5000) 20/20 
 [=================================================================================>....]  Step: 33ms | Tot: 1s54ms | Loss: 7.487 | Acc: 21.340% (1067/5000) 20/20 
Saving..

Epoch: 3
 [=================================================================================>....]  Step: 106ms | Tot: 3s167ms | Loss: 2.

In [0]:
#Test baseline

baseline_model = torch.load('./models/baseline_train.pth')

baseline_model.eval()
test_loss, test_acc = validate(testloader, baseline_model, epoch)
print('Baseline test output -')
print('test loss = ', str(test_loss))
print('test accuracy = ', str(test_acc))

train.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


 [=================================================================================>....]  Step: 29ms | Tot: 1s82ms | Loss: 2.474 | Acc: 65.480% (3274/5000) 20/20 
Saving..
Baseline test output -
test loss =  tensor(2.6042, device='cuda:0')
test accuracy =  tensor(65.4800)


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DataParallel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python

In [0]:
! ls ./results

In [0]:
#Mixup
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.t7' + args.name + '_'
                            + str(args.seed))
    mixup_net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch'] + 1
    rng_state = checkpoint['rng_state']
    torch.set_rng_state(rng_state)
else:
    print('==> Building model..')
    mixup_net = models.resnet18()

if not os.path.isdir('results'):
    os.mkdir('results')
logname = ('results/log_mixup' + mixup_net.__class__.__name__ + '_' + args.name + '_'
           + str(args.seed) + '.csv')

if use_cuda:
    mixup_net.cuda()
    mixup_net = torch.nn.DataParallel(mixup_net)
    print(torch.cuda.device_count())
    cudnn.benchmark = True
    print('Using CUDA..')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mixup_net.parameters(), lr=args.lr, momentum=0.9,
                      weight_decay=args.decay)


def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    loss = lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)
    return loss

def mixup_train(epoch):
    print('\nEpoch: %d' % epoch)
    mixup_net.train()
    train_loss = 0
    reg_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader_labelled):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()

        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets,
                                                       args.alpha, use_cuda)
        inputs, targets_a, targets_b = map(Variable, (inputs,
                                                      targets_a, targets_b))
        outputs = mixup_net(inputs)
        loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
        train_loss += loss.data
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (lam * predicted.eq(targets_a.data).cpu().sum().float()
                    + (1 - lam) * predicted.eq(targets_b.data).cpu().sum().float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        progress_bar(batch_idx, len(trainloader_labelled),
                     'Loss: %.3f | Reg: %.5f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), reg_loss/(batch_idx+1),
                        100.*correct/total, correct, total))
    return (train_loss/batch_idx, reg_loss/batch_idx, 100.*correct/total)

if not os.path.exists(logname):
  with open(logname, 'w') as logfile:
      logwriter = csv.writer(logfile, delimiter=',')
      logwriter.writerow(['epoch', 'train loss', 'reg loss', 'train acc', 'mixup train loss', 'mixup reg loss', 'mixup train acc',
                          'test loss', 'test acc'])

#mixup train
for epoch in range(start_epoch, args.epoch):
    train_loss, reg_loss, train_acc = baseline_train(mixup_net, epoch)
    m_train_loss, m_reg_loss, m_train_acc = mixup_train(epoch)
    test_loss, test_acc = validate(valloader, mixup_net, epoch)
    adjust_learning_rate(optimizer, epoch)
    with open(logname, 'a') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow([epoch, train_loss, reg_loss, train_acc, m_train_loss, m_reg_loss, m_train_acc, test_loss,
                            test_acc])

if not os.path.isdir('models'):
    os.mkdir('models')
torch.save(mixup_net, './models/mixup_train.pth')

==> Building model..
1
Using CUDA..

Epoch: 0
 [=================================================================================>....]  Step: 103ms | Tot: 3s87ms | Loss: 0.013 | Reg: 0.00000 | Acc: 99.580% (4979/5000) 20/20 

Epoch: 0
 [=================================================================================>....]  Step: 103ms | Tot: 3s100ms | Loss: 4.452 | Reg: 0.00000 | Acc: 11.068% (553/5000) 20/20 


train.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


 [=================================================================================>....]  Step: 34ms | Tot: 1s18ms | Loss: 508.359 | Acc: 10.060% (503/5000) 20/20 
Saving..

Epoch: 1


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DataParallel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python

 [=================================================================================>....]  Step: 103ms | Tot: 3s98ms | Loss: 0.012 | Reg: 0.00000 | Acc: 99.720% (4986/5000) 20/20 

Epoch: 1
 [=================================================================================>....]  Step: 98ms | Tot: 3s347ms | Loss: 3.117 | Reg: 0.00000 | Acc: 13.297% (664/5000) 20/20 
 [=================================================================================>....]  Step: 28ms | Tot: 1s88ms | Loss: 4.737 | Acc: 12.520% (626/5000) 20/20 
Saving..

Epoch: 2
 [=================================================================================>....]  Step: 104ms | Tot: 3s166ms | Loss: 0.015 | Reg: 0.00000 | Acc: 99.560% (4978/5000) 20/20 

Epoch: 2
 [=================================================================================>....]  Step: 107ms | Tot: 

In [0]:
#Test mixup

mixup_model = torch.load('./models/mixup_train.pth')
mixup_model.eval()
test_loss, test_acc = validate(testloader, mixup_model, epoch)
print('Mixup test output -')
print('test loss = ', str(test_loss))
print('test accuracy = ', str(test_acc))

train.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


 [=================================================================================>....]  Step: 31ms | Tot: 1s242ms | Loss: 1.034 | Acc: 66.280% (3314/5000) 20/20 
Saving..
Mixup test output -
test loss =  tensor(1.0881, device='cuda:0')
test accuracy =  tensor(66.2800)


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DataParallel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python

In [0]:
#Cutmix
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.t7' + args.name + '_'
                            + str(args.seed))
    cutmix_net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch'] + 1
    rng_state = checkpoint['rng_state']
    torch.set_rng_state(rng_state)
else:
    print('==> Building model..')
    cutmix_net = models.resnet18()

if not os.path.isdir('results'):
    os.mkdir('results')
logname = ('results/log_cutmix' + cutmix_net.__class__.__name__ + '_' + args.name + '_'
           + str(args.seed) + '.csv')

if use_cuda:
    cutmix_net.cuda()
    cutmix_net = torch.nn.DataParallel(cutmix_net)
    print(torch.cuda.device_count())
    cudnn.benchmark = True
    print('Using CUDA..')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cutmix_net.parameters(), lr=args.lr, momentum=0.9,
                      weight_decay=args.decay)

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix_train(epoch):
    print('\nEpoch: %d' % epoch)
    cutmix_net.train()
    train_loss = 0
    reg_loss = 0
    correct = 0
    total = 0

    for i, (inputs, targets) in enumerate(trainloader_labelled):

      inputs = inputs.cuda()
      targets = targets.cuda()

      r = np.random.rand(1)
      if args.beta > 0 and r < args.cutmix_prob:
          
          # generate mixed sample
          lam = np.random.beta(args.beta, args.beta)
          rand_index = torch.randperm(inputs.size()[0]).cuda()
          target_a = targets
          target_b = targets[rand_index]
          bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
          inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
          
          # adjust lambda to exactly match pixel ratio
          lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
          
          # compute output
          input_var = torch.autograd.Variable(inputs, requires_grad=True)
          target_a_var = torch.autograd.Variable(target_a)
          target_b_var = torch.autograd.Variable(target_b)
          output = cutmix_net(input_var)
          loss = criterion(output, target_a_var) * lam + criterion(output, target_b_var) * (1. - lam)
          _, predicted = torch.max(output.data, 1)
          correct += predicted.eq(targets.data).cpu().sum()
      else:
          # compute output
          input_var = torch.autograd.Variable(inputs, requires_grad=True)
          target_var = torch.autograd.Variable(targets)
          output = cutmix_net(input_var)
          loss = criterion(output, target_var)
          _, predicted = torch.max(output.data, 1)
          correct += predicted.eq(targets.data).cpu().sum()
          

      train_loss += loss.data
      total += targets.size(0)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      progress_bar(i, len(trainloader_labelled),
                    'Loss: %.3f | Reg: %.5f | Acc: %.3f%% (%d/%d)'
                    % (train_loss/(i+1), reg_loss/(i+1),
                      100.*correct/total, correct, total))
    return (train_loss/i, reg_loss/i, 100.*correct/total)

if not os.path.exists(logname):
    with open(logname, 'w') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow(['epoch', 'train loss', 'reg loss', 'train acc', 'cutmix train loss', 'cutmix reg loss', 'cutmix train acc',
                            'test loss', 'test acc'])

for epoch in range(start_epoch, args.epoch):
    train_loss, reg_loss, train_acc = baseline_train(cutmix_net, epoch)
    c_train_loss, c_reg_loss, c_train_acc = cutmix_train(epoch)
    test_loss, test_acc = validate(valloader, cutmix_net, epoch)
    adjust_learning_rate(optimizer, epoch)
    with open(logname, 'a') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow([epoch, train_loss, reg_loss, train_acc, c_train_loss, c_reg_loss, c_train_acc, test_loss,
                            test_acc])
        
if not os.path.isdir('models'):
    os.mkdir('models')
torch.save(cutmix_net, './models/cutmix_train.pth')

==> Building model..
1
Using CUDA..

Epoch: 0
 [=================================================================================>....]  Step: 102ms | Tot: 3s173ms | Loss: 0.015 | Reg: 0.00000 | Acc: 99.560% (4978/5000) 20/20 

Epoch: 0
 [=================================================================================>....]  Step: 134ms | Tot: 3s303ms | Loss: 5.734 | Reg: 0.00000 | Acc: 11.400% (570/5000) 20/20 


train.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


 [=================================================================================>....]  Step: 27ms | Tot: 1s54ms | Loss: 2525.705 | Acc: 9.340% (467/5000) 20/20 
Saving..

Epoch: 1


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DataParallel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python

 [=================================================================================>....]  Step: 103ms | Tot: 3s127ms | Loss: 0.013 | Reg: 0.00000 | Acc: 99.620% (4981/5000) 20/20 

Epoch: 1
 [=================================================================================>....]  Step: 105ms | Tot: 3s143ms | Loss: 4.308 | Reg: 0.00000 | Acc: 12.540% (627/5000) 20/20 
 [=================================================================================>....]  Step: 35ms | Tot: 1s58ms | Loss: 3.158 | Acc: 16.780% (839/5000) 20/20 
Saving..

Epoch: 2
 [=================================================================================>....]  Step: 100ms | Tot: 3s23ms | Loss: 0.013 | Reg: 0.00000 | Acc: 99.580% (4979/5000) 20/20 

Epoch: 2
 [=================================================================================>....]  Step: 108ms | Tot:

In [0]:
#Test cutmix

cutmix_model = torch.load('./models/cutmix_train.pth')
cutmix_model.eval()
test_loss, test_acc = validate(testloader, cutmix_model, epoch)
print('Cutmix test output -')
print('test loss = ', str(test_loss))
print('test accuracy = ', str(test_acc))

train.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


 [=================================================================================>....]  Step: 36ms | Tot: 1s248ms | Loss: 0.927 | Acc: 69.760% (3488/5000) 20/20 
Saving..
Cutmix test output -
test loss =  tensor(0.9762, device='cuda:0')
test accuracy =  tensor(69.7600)


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DataParallel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python

In [0]:
! ls ./models

baseline_train.pth  cutmix_train.pth  mixup_train.pth


In [0]:
from google.colab import files
#files.download('./models/baseline_train.pth')
#files.download('./models/cutmix_train.pth')
files.download('./models/mixup_train.pth')

In [0]:
#Entropy Minimization
import torch.nn.functional as F
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.t7' + args.name + '_'
                            + str(args.seed))
    net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch'] + 1
    rng_state = checkpoint['rng_state']
    torch.set_rng_state(rng_state)
else:
    print('==> Building model..')
    net = models.resnet18()

if not os.path.isdir('results'):
    os.mkdir('results')
logname = ('results/log_entropy_min' + net.__class__.__name__ + '_' + args.name + '_'
           + str(args.seed) + '.csv')

if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net)
    print(torch.cuda.device_count())
    cudnn.benchmark = True
    print('Using CUDA..')

class HLoss(nn.Module):
    def __init__(self):
        super(HLoss, self).__init__()

    def forward(self, x):
        b = F.softmax(x, dim=1) * F.log_softmax(x, dim=1)
        b = -1.0 * b.sum()
        return b

criterion = nn.CrossEntropyLoss()        
criterion2 = HLoss()

optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9,
                      weight_decay=args.decay)

q = int(50000*percentage/25600)

def EM_train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    reg_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        outputs = net(inputs)
        if batch_idx <= q :        
          loss = criterion(outputs, targets)
        else :
          loss = criterion2(outputs)
        train_loss += loss.data
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progress_bar(batch_idx, len(trainloader),
                      'Loss: %.3f |Acc: %.3f%% (%d/%d)'
                      % (train_loss/(batch_idx+1), 
                        100.*correct/total, correct, total))
        

    '''for batch_idx, (inputs, targets) in enumerate(trainloader_labelled):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        train_loss += loss.data
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progress_bar(batch_idx, len(trainloader_labelled),
                      'Loss: %.3f |Acc: %.3f%% (%d/%d)'
                      % (train_loss/(batch_idx+1), 
                        100.*correct/total, correct, total))
        
    for batch_idx, (inputs, targets) in enumerate(trainloader_unlabelled):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()        
        outputs = net(inputs)
        e_loss = criterion2(outputs)
        e_loss = e_loss * 0.1
        train_loss += e_loss.data
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        optimizer.zero_grad()
        e_loss.backward()       
        optimizer.step()
        progress_bar(batch_idx, len(trainloader_unlabelled),
                     'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1),
                        100.*correct/total, correct, total))'''
        
    return (train_loss/batch_idx, reg_loss/batch_idx, 100.*correct/total)



if not os.path.exists(logname):
    with open(logname, 'w') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow(['epoch', 'train loss', 'reg loss', 'train acc',
                            'test loss', 'test acc'])

for epoch in range(start_epoch, args.epoch):
    train_loss, reg_loss, train_acc = EM_train(epoch)
    test_loss, test_acc = val_test(epoch)
    adjust_learning_rate(optimizer, epoch)
    with open(logname, 'a') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow([epoch, train_loss, reg_loss, train_acc, test_loss,
                            test_acc])

if not os.path.isdir('models'):
    os.mkdir('models')
torch.save(net.state_dict(), 'models/log_entropy_min')

The above implementation is adapted from -

[mixup-cifar10](https://https://github.com/facebookresearch/mixup-cifar10)
[CutMix-PyTorch](https://https://github.com/clovaai/CutMix-PyTorch)